In [1]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
to_fit = pd.read_csv("Data/costs_input_new.csv")

In [3]:
to_fit.head()

,Variable Name,Value,Lower Bound,Upper Bound
0,cLevel1NurseryAdmissionPerDay,703.0,234.0,1172.0
1,cLevel1NursingCarePerDay,410.9,329.0,493.0
2,cLevel2NICUAdmissionPerDay,3066.0,1022.0,5109.0
3,cLevel2NursingCarePerDay,501.1,401.0,601.0
4,cGentamicin1,21.0,7.0,34.0


In [4]:
to_fit.tail()

,Variable Name,Value,Lower Bound,Upper Bound
8,cCSF,104.0,35.0,174.00
9,cReadmission,5378.5,4302.8,6454.19
10,cOtotoxicity,520114.0,144741.0,875120.00
11,cNeurocognitiveDisability,1137012.0,875120.0,1709879.00
12,cInfantDeath,10194674.0,5825528.0,14563822.00


In [5]:
to_fit = to_fit[0:9]

In [6]:
to_fit.tail()

,Variable Name,Value,Lower Bound,Upper Bound
4,cGentamicin1,21.0,7.0,34.0
5,cAmpicillin1,60.0,20.0,100.0
6,cInitialNICUTests,682.0,227.0,1137.0
7,cBloodCulture1,172.0,57.0,287.0
8,cCSF,104.0,35.0,174.0


In [7]:
# fits an encompassing gamma distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_gamma_distr(value, lower_bound, upper_bound):

    a_lower_bound = 100
    a_upper_bound = 75
    linspace_points = 201
    search_range = np.linspace(a_lower_bound, a_upper_bound, linspace_points)

    for a in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=a, scale=value / a)
        ppf_025 = ss.gamma.ppf(0.025, a=a, scale=value / a)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            scale = value / a
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    a_lower_bound = 6
    a_upper_bound = 5
    linspace_points = 1001
    search_range = np.linspace(a_lower_bound, a_upper_bound, linspace_points)

    for a in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=a, scale=value / a)
        ppf_025 = ss.gamma.ppf(0.025, a=a, scale=value / a)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            scale = value / a
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    print("No match found")
    return 0, 0

In [8]:
to_fit["a"], to_fit["scale"] = np.vectorize(fit_encompassing_gamma_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

a: 5.18 	 scale: 135.77
a: 5.18 	 scale: 135.77
a: 86.62 	 scale: 4.74
a: 5.19 	 scale: 590.98
a: 86.25 	 scale: 5.81
a: 5.19 	 scale: 4.05
a: 5.19 	 scale: 11.57
a: 5.18 	 scale: 131.74
a: 5.14 	 scale: 33.43
a: 5.26 	 scale: 19.77


In [9]:
to_fit["gamma_mean"] = to_fit["a"] * to_fit["scale"]
to_fit["gamma_2.5pct"] = ss.gamma.ppf(0.025, a=to_fit["a"], scale=to_fit["scale"])
to_fit["gamma_97.5pct"] = ss.gamma.ppf(0.975, a=to_fit["a"], scale=to_fit["scale"])

In [10]:
to_fit

,Variable Name,Value,Lower Bound,Upper Bound,a,scale,gamma_mean,gamma_2.5pct,gamma_97.5pct
0,cLevel1NurseryAdmissionPerDay,703.0,234.0,1172.0,5.178,135.766705,703.0,233.999288,1425.361305
1,cLevel1NursingCarePerDay,410.9,329.0,493.0,86.625,4.743434,410.9,328.947266,501.830761
2,cLevel2NICUAdmissionPerDay,3066.0,1022.0,5109.0,5.188,590.979183,3066.0,1021.921603,6212.966479
3,cLevel2NursingCarePerDay,501.1,401.0,601.0,86.250,5.809855,501.1,400.952481,612.243949
4,cGentamicin1,21.0,7.0,34.0,5.188,4.047803,21.0,6.999463,42.554565
5,cAmpicillin1,60.0,20.0,100.0,5.188,11.565150,60.0,19.998466,121.584471
6,cInitialNICUTests,682.0,227.0,1137.0,5.177,131.736527,682.0,226.978569,1382.860356
7,cBloodCulture1,172.0,57.0,287.0,5.145,33.430515,172.0,56.995276,349.384527
8,cCSF,104.0,35.0,174.0,5.260,19.771863,104.0,34.997852,209.908595


In [11]:
to_fit.to_csv("Data/costs_output_new_2.csv")